In [1]:
import os
import qrcode
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from qrcode.image.styledpil import StyledPilImage
from qrcode.image.styles.moduledrawers.pil import RoundedModuleDrawer
import pywhatkit as kit
import time

In [6]:
def load_excel():
    excel_file_path = "tabela/tabela.xlsx"
    df = pd.read_excel(
        excel_file_path,
        dtype={"cpf": str, "nome": str, "presença": str, "telefone": str}
    )
    
    cpf = df.iloc[0]["cpf"]
    name = df.iloc[0]["nome"]
    telefone = df.iloc[0]["telefone"]

    return cpf, name, telefone

In [49]:
def generate_qr_code(data):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=17,
        border=1,
    )
    qr.add_data(data)
    qr.make(fit=True)

    # img = qr.make_image(fill_color=fill_color, back_color=back_color)
    img = qr.make_image(
        fill_color="black",
        back_color="white",
        image_factory=StyledPilImage,
        module_drawer=RoundedModuleDrawer(radius_ratio=0.7)
        # color_mask=RadialGradiantColorMask()
        # embeded_image_path="/path/to/image.png"
    )
    return img

def gerando_automatico():
    df = pd.read_excel("tabela/tabela.xlsx")
    
    
    for index, row in df.iterrows():
        codigo_qr = row['codigo']
        qr_image = generate_qr_code(codigo_qr)
        qr_image.save(f"qrcodes/qrcode/qr_{codigo_qr}.png")
        
gerando_automatico()

In [71]:
def insert_qr_code(qr_image_path, qr_dados_path):
    base_image_path = "assets/imagem-convite.png"
    
    base_image = Image.open(base_image_path)
    qr_image = Image.open(qr_image_path)
    qr_dados = Image.open(qr_dados_path)
    
    base_width, base_height = base_image.size
    qr_width, qr_height = qr_image.size

    position1 = ((base_width - qr_width) - 890, base_height - qr_height - 10)
    position2 = ((base_width - qr_width) - 300, base_height - qr_height + 150)

    # Inserir o QR code na posição especificada
    base_image.paste(qr_image, position1)
    base_image.paste(qr_dados, position2)

    # Salvar a imagem final
    base_image.save("final_image.png")

# Caminho para a imagem base e QR code gerado
qr_image_path = "qrcodes/qrcode/qr_10000.png"
qr_dados_path = "qrcodes/dados/dados_10000.png"



insert_qr_code(base_image_path, qr_image_path, qr_dados_path, position1, position2)

In [ ]:
# Função para gerar um QR code personalizado
def generate_qr_code(data, fill_color="black", back_color="white"):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=10,
        border=1,
    )
    qr.add_data(data)
    qr.make(fit=True)

    img = qr.make_image(fill_color=fill_color, back_color=back_color).convert('RGBA')
    return img

# Função para inserir o QR code e texto em uma imagem de fundo
def insert_qr_code_and_text(base_image_path, qr_image, text, qr_position, text_position, font_path=None, font_size=20):
    base_image = Image.open(base_image_path).convert('RGBA')
    draw = ImageDraw.Draw(base_image)

    # Inserir o QR code na posição especificada
    base_image.paste(qr_image, qr_position, qr_image)

    # Carregar a fonte da pasta raiz do projeto
    if font_path:
        try:
            font = ImageFont.truetype(font_path, font_size)
        except IOError:
            print(f"Não foi possível carregar a fonte {font_path}. Usando a fonte padrão.")
            font = ImageFont.load_default()
    else:
        font = ImageFont.load_default()

    # Inserir o texto na posição especificada
    for i, line in enumerate(text.split('\n')):
        draw.text((text_position[0], text_position[1] + i * font_size), line, font=font, fill="black")

    # Salvar a imagem final
    base_image.save("final_image.png")

# Caminho para a imagem base e QR code gerado
base_image_path = "imagem-convite.png"  # Substitua pelo caminho da sua imagem de fundo

# Caminho para a fonte TrueType
font_path = os.path.join(os.getcwd(), "assets/Roboto-Regular.ttf")

# Ler os dados do Excel
df = pd.read_excel("tabela/tabela.xlsx")

# Definir o tamanho da fonte
font_size = 40

# Iterar sobre cada linha do DataFrame
for index, row in df.iterrows():
    cpf = row['cpf']
    nome = row['nome']
    telefone = row['telefone']
    
    # Gerar o QR code
    data = f"Nome: {nome}\nCPF: {cpf}\nTelefone: {telefone}"
    qr_image = generate_qr_code(data, fill_color="blue", back_color="white")
    
    # Calcular a posição do QR code e texto
    base_image = Image.open(base_image_path)
    base_width, base_height = base_image.size
    qr_width, qr_height = qr_image.size
    
    # Posições
    qr_position = (base_width - qr_width - 10, base_height - qr_height - 10)  # QR code no canto inferior direito
    text_position = (qr_position[0] - 700, qr_position[1] + 150)  # Texto à esquerda do QR code, com uma largura máxima de 200 pixels

    # Inserir o QR code e o texto na imagem de fundo
    text = f"CPF: {cpf}\nNome: {nome}\nTelefone: {telefone}"
    insert_qr_code_and_text(base_image_path, qr_image, text, qr_position, text_position, font_path=font_path, font_size=font_size)

    # Renomear a imagem final para evitar sobrescrever
    output_image_path = f"qrcode/final_image_{index}.png"
    Image.open("final_image.png").save(output_image_path)
    print(f"Imagem salva como {output_image_path}")

In [ ]:
# Função para criar uma imagem com texto centralizado a partir dos dados do Excel

def create_centered_text_image(text, image_size, font_path=None, font_size=24, output_image_path=output_image_path):
    # Criar uma imagem branca
    image = Image.new('RGBA', image_size, "white")
    draw = ImageDraw.Draw(image)

    font_path = os.path.join(os.getcwd(), "assets/Roboto-Regular.ttf")

    # Carregar a fonte da pasta raiz do projeto
    if font_path:
        try:
            font = ImageFont.truetype(font_path, font_size)
        except IOError:
            print(f"Não foi possível carregar a fonte {font_path}. Usando a fonte padrão.")
            font = ImageFont.load_default()
    else:
        font = ImageFont.load_default()

    # Calcular o tamanho do texto e a posição centralizada
    text_lines = text.split('\n')
    line_height = font_size + 5  # Adiciona um pequeno espaçamento entre linhas
    total_text_height = line_height * len(text_lines)

    y_start = (image_size[1] - total_text_height) // 2
    for i, line in enumerate(text_lines):
        text_width = draw.textbbox((0, 0), line, font=font)[2]  # Pega a largura do texto
        x_position = (image_size[0] - text_width) // 2
        y_position = y_start + i * line_height
        draw.text((x_position, y_position), line, font=font, fill="black")

    # Salvar a imagem final
    image.save(output_image_path)


# Ler os dados do Excel
df = pd.read_excel("tabela/tabela.xlsx")

# Definir o tamanho da fonte e o tamanho da imagem
font_size = 32
image_width = 600
image_height = 180

# Iterar sobre cada linha do DataFrame e gerar uma imagem para cada linha
for index, row in df.iterrows():
    codigo_qr = row['codigo']
    cpf = row['cpf']
    nome = row['nome']
    telefone = row['telefone']
    text = f"Nome: {nome}\nCPF: {cpf}\nTelefone: {telefone}"
    
    # Criar a imagem com o texto centralizado
    output_image_path = f"qrcodes/dados/dados_{codigo_qr}.png"
    create_centered_text_image(text, (image_width, image_height), font_path=font_path, font_size=font_size, output_image_path=output_image_path)

    print(f"Imagem salva como {output_image_path}")


In [47]:
import time
import openpyxl
import urllib.parse
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton

In [17]:
options = Options()
options.add_argument("start-maximized")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://web.whatsapp.com")

while len(driver.find_elements(by=By.ID, value='side')) < 1:
    time.sleep(1)
time.sleep(2)

In [ ]:
path_lista = "tabela/lista-teste.xlsx"
        
df = pd.read_excel(path_lista)
wb = openpyxl.load_workbook(path_lista)
ws = wb.active

print('Lista aceita')
display(df)

In [ ]:
try:
    indice = df[df['sequencia'] != '-'].iloc[-1]['sequencia']
    sequencia = indice
except:
    indice = 0
    sequencia = 0

for i in range(indice, len(df)):
    index = df.loc[i, "indice"]
    nome = df.loc[i, "nome"]
    celular = df.loc[i, "celular"]
    dataAtualCobranca = datetime.now().strftime("%d/%m/%Y %H:%M")
    status_200 = "Ok!"
    status_400 = "Inválido!"
    status_404 = "Erro!"


    send_text = f'''
    Olá *{nome}* 🤩
    Seu convite do *Arráia Dusnei* CHEGOU!!! 🎉
    '''

    time.sleep(2)
    
    texto_codificado = urllib.parse.quote(send_text)
    link = f"https://web.whatsapp.com/send?phone={celular}&text={texto_codificado}"
    driver.get(link)

    time.sleep(4)
    
    try:
        while len(driver.find_elements(by=By.ID, value='side')) < 1:
            print("Aguardando")
            time.sleep(3)
    except:
        while len(driver.find_elements(by=By.ID, value='side')) < 1:
            time.sleep(3)
        
    if len(driver.find_elements(by=By.XPATH, value='//*[@id="app"]')) > 0:
        time.sleep(1)
        driver.find_elements(by=By.XPATH , value='//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/div/div/span').click()

In [62]:
time.sleep(5)

driver.find_elements(by=By.CLASS_NAME, value='selectable-text').click()

AttributeError: 'list' object has no attribute 'click'